In [4]:
# Getting a list of Nightclubs in San Francisco
# We want to demostrate the use of APIs

# Dependencies
import requests
import json
import os
import gmaps
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd
import requests
import time

# Google  API key
from config import gkey

gmaps.configure(api_key=gkey)

In [5]:
# Getting a list of Nightclubs in San Francisco
# We want to demostrate the use of APIs

In [6]:
target_coordinates = "37.7749,-122.4194"
target_search = "Nightclub"
target_radius = 50000
target_type = "nightclub"

# Setting up parameters
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [7]:
# convert response to json
sf_clubs = response.json()

# Print the json (pretty printed)
print(json.dumps(sf_clubs, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "next_page_token": "CrQCKwEAAI8_7HttsefO5tpxhIlFLOhojSgjCfAE0IEh-CzbZrXDAkxresof5H1tegbM_LmOso7rENSmkGQ699UcPwbqS7v1SPxZzX7Gz2iaoohuPDDYxcEC6kXDl9cD3iz9UnyFW7CbMMABFefIKr0aE3FzbuOtV6dwMhQb22WQZ8FRjvgwOTLMOUDaNl7whiyXXfz6LX5gN_IoK8JjQfuC6JSpS7oc3Ht5WAwQQPjohp1fb4sClKp8B6R6aH6WI1EjQjgugCVTFWdI7e648241J9S8g04Ppaeul5Hsf7WD984pZz10-AqjNY3zhsiJQnQLVH_lG8sAUIlmcfo7Fg4WAMJ1NzywZPGMgwdIBasgT477568qf00NEiV4xaxM7-5xikZ-y1p_89C4MYyjGjhzc0WJrvgSEJKgJPPvrB_NFJ0ifap8JkcaFBe1QmfG2RaaaCpLF9P_N5iRbCSX",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 37.3768037,
                    "lng": -122.0298038
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.37818862989272,
                        "lng": -122.0286148701073
                    },
                    "southwest": {
                        "lat": 37.37548897010728,
                    

In [8]:
print(len(sf_clubs["results"]))

20


In [9]:
name=[]
vicinity =[]
lat = []
lng =[]
i= 0
while i < 20:
        name.append(sf_clubs["results"][i]["name"])
        vicinity.append((sf_clubs["results"][i]["vicinity"]))
        lat.append(sf_clubs["results"][i]['geometry']['location']['lat'])
        lng.append(sf_clubs["results"][i]['geometry']['location']['lng'])
        i+=1

clubs_dict = {
    "Club Name": name
    , "Address": vicinity
    , "Latitude": lat
    , "Longitude": lng
}
clubs_data = pd.DataFrame(clubs_dict)
clubs_data

,Club Name,Address,Latitude,Longitude
0,Pure Nightclub,"146 S Murphy Ave, Sunnyvale",37.376804,-122.029804
1,HUE,"447 Broadway, San Francisco",37.797948,-122.404785
2,The Holy Cow,"1535 Folsom St, San Francisco",37.771375,-122.414310
3,Origin,"1538 Fillmore St, San Francisco",37.783884,-122.432553
4,F8,"1192 Folsom St, San Francisco",37.775291,-122.409961
5,George's Nightclub,"842 4th St, San Rafael",37.972878,-122.524798
6,The Grand,"520 4th St, San Francisco",37.779100,-122.397900
7,Audio Nightclub,"316 11th St, San Francisco",37.771362,-122.413792
8,1015 Folsom,"1015 Folsom St, San Francisco",37.778127,-122.405787
9,Harlot,"46 Minna St, San Francisco",37.788569,-122.398197


In [10]:
crime=pd.read_csv("sf_crime.csv")

In [11]:
crime.dropna()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,1/29/2016 0:00,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",1.200580e+13
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,1/29/2016 0:00,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",1.200580e+13
2,141059263,WARRANTS,WARRANT ARREST,Monday,4/25/2016 0:00,14:59,BAYVIEW,"ARREST, BOOKED",KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",1.410590e+13
3,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,1/5/2016 0:00,23:50,TENDERLOIN,NONE,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)",1.600140e+13
4,160002740,NON-CRIMINAL,LOST PROPERTY,Friday,1/1/2016 0:00,0:30,MISSION,NONE,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)",1.600030e+13
5,160002869,ASSAULT,BATTERY,Friday,1/1/2016 0:00,21:35,NORTHERN,NONE,1700 Block of BUSH ST,-122.426077,37.788019,"(37.788018555829, -122.426077177375)",1.600030e+13
6,160003130,OTHER OFFENSES,PAROLE VIOLATION,Saturday,1/2/2016 0:00,0:04,SOUTHERN,"ARREST, BOOKED",MARY ST / HOWARD ST,-122.405722,37.780879,"(37.7808789360214, -122.405721454567)",1.600030e+13
7,160003259,NON-CRIMINAL,FIRE REPORT,Saturday,1/2/2016 0:00,1:02,TENDERLOIN,NONE,200 Block of EDDY ST,-122.411778,37.783981,"(37.7839805592634, -122.411778295992)",1.600030e+13
8,160003970,WARRANTS,WARRANT ARREST,Saturday,1/2/2016 0:00,12:21,SOUTHERN,"ARREST, BOOKED",4TH ST / BERRY ST,-122.393357,37.775788,"(37.7757876218293, -122.393357241451)",1.600040e+13
9,160003641,MISSING PERSON,FOUND PERSON,Friday,1/1/2016 0:00,10:06,BAYVIEW,NONE,100 Block of CAMERON WY,-122.387182,37.720967,"(37.7209669615499, -122.387181635995)",1.600040e+13


In [12]:
for col in crime.columns:
    print(col)

IncidntNum
Category
Descript
DayOfWeek
Date
Time
PdDistrict
Resolution
Address
X
Y
Location
PdId
